## Cleaning the Data

In [189]:
##importing the necessary packages
import pandas as pd
import numpy as np
import math
import scipy.optimize as opt
from scipy.optimize import minimize
import scipy.stats as stats
import time
from statsmodels.iolib.summary2 import summary_col

In [190]:
#importing the data and dropping missing values
df1 = pd.read_stata('/Users/alexandradinu/Desktop/CompEcon_Fall17/Functions/PS3_data.dta')
df1 = df1[df1.hlabinc.isnull() !=True] #drop if missing hlabinc
df1 = df1[df1.hannhrs.isnull() !=True] #drop if missing hannhrs
df1 = df1[df1.age.isnull() !=True] #drop if missing age
df1 = df1[df1.hyrsed.isnull() !=True] #drop if missing educ

In [191]:
df1.head()

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
11161,402,1971,1,Head,1523.0,0.0,62928.707031,NaN,0,1.0,...,15801.0,1.0,2.0,51.0,48.0,1.0,2.0,12.0,12.0,0.247121
11162,446,1971,2,Head,520.0,0.0,1618.640747,NaN,0,NaN,...,5153.0,2.0,NaN,62.0,NaN,1.0,NaN,17.0,NaN,0.247121
11164,461,1971,4,Head,2010.0,0.0,22660.970703,NaN,0,1.0,...,5600.0,1.0,2.0,55.0,54.0,1.0,2.0,5.0,5.0,0.247121
11165,462,1971,5,Head,1960.0,0.0,12949.125977,NaN,0,NaN,...,3200.0,1.0,2.0,59.0,55.0,1.0,2.0,5.0,8.0,0.247121
11166,1126,1971,8,Head,2860.0,0.0,29337.865234,NaN,1,NaN,...,7250.0,1.0,2.0,25.0,24.0,1.0,2.0,16.0,12.0,0.247121


In [192]:
#Renaming the variable for education "Educ"
df1=df1.rename(columns= {'hyrsed': 'Educ'})

In [193]:
#Creating race dummies
df1_hrace =pd.get_dummies(df1['hrace'], prefix='race')

In [194]:
df_new = pd.concat([df1, df1_hrace], axis=1)

In [195]:
df_new

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,age,wage,hpersno,wpersno,Educ,wyrsed,pce,race_1.0,race_2.0,race_3.0
11161,402,1971,1,Head,1523.0,0.0,62928.707031,NaN,0,1.0,...,51.0,48.0,1.0,2.0,12.0,12.0,0.247121,1,0,0
11162,446,1971,2,Head,520.0,0.0,1618.640747,NaN,0,NaN,...,62.0,NaN,1.0,NaN,17.0,NaN,0.247121,1,0,0
11164,461,1971,4,Head,2010.0,0.0,22660.970703,NaN,0,1.0,...,55.0,54.0,1.0,2.0,5.0,5.0,0.247121,1,0,0
11165,462,1971,5,Head,1960.0,0.0,12949.125977,NaN,0,NaN,...,59.0,55.0,1.0,2.0,5.0,8.0,0.247121,1,0,0
11166,1126,1971,8,Head,2860.0,0.0,29337.865234,NaN,1,NaN,...,25.0,24.0,1.0,2.0,16.0,12.0,0.247121,1,0,0
11167,1585,1971,10,Head,840.0,833.0,8627.355469,6628.333984,0,1.0,...,67.0,66.0,1.0,2.0,12.0,12.0,0.247121,1,0,0
11170,97,1971,15,Head,1960.0,0.0,19678.625000,NaN,0,1.0,...,63.0,64.0,1.0,2.0,6.0,8.0,0.247121,1,0,0
11171,237,1971,16,Head,604.0,0.0,2124.466064,NaN,0,NaN,...,65.0,NaN,1.0,NaN,8.0,NaN,0.247121,1,0,0
11172,669,1971,17,Head,683.0,0.0,3439.611816,NaN,0,NaN,...,66.0,NaN,2.0,NaN,12.0,NaN,0.247121,1,0,0
11173,284,1971,20,Head,2400.0,0.0,76885.437500,NaN,2,1.0,...,39.0,36.0,1.0,2.0,16.0,12.0,0.247121,1,0,0


In [196]:
# Renaming the race dummies that I will actually use
df_new=df_new.rename(columns= {'race_1.0': 'White'})
df_new=df_new.rename(columns= {'race_2.0': 'Black'})
df_new=df_new.rename(columns= {'race_3.0': 'Other'})

In [197]:
df_new

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,age,wage,hpersno,wpersno,Educ,wyrsed,pce,White,Black,Other
11161,402,1971,1,Head,1523.0,0.0,62928.707031,NaN,0,1.0,...,51.0,48.0,1.0,2.0,12.0,12.0,0.247121,1,0,0
11162,446,1971,2,Head,520.0,0.0,1618.640747,NaN,0,NaN,...,62.0,NaN,1.0,NaN,17.0,NaN,0.247121,1,0,0
11164,461,1971,4,Head,2010.0,0.0,22660.970703,NaN,0,1.0,...,55.0,54.0,1.0,2.0,5.0,5.0,0.247121,1,0,0
11165,462,1971,5,Head,1960.0,0.0,12949.125977,NaN,0,NaN,...,59.0,55.0,1.0,2.0,5.0,8.0,0.247121,1,0,0
11166,1126,1971,8,Head,2860.0,0.0,29337.865234,NaN,1,NaN,...,25.0,24.0,1.0,2.0,16.0,12.0,0.247121,1,0,0
11167,1585,1971,10,Head,840.0,833.0,8627.355469,6628.333984,0,1.0,...,67.0,66.0,1.0,2.0,12.0,12.0,0.247121,1,0,0
11170,97,1971,15,Head,1960.0,0.0,19678.625000,NaN,0,1.0,...,63.0,64.0,1.0,2.0,6.0,8.0,0.247121,1,0,0
11171,237,1971,16,Head,604.0,0.0,2124.466064,NaN,0,NaN,...,65.0,NaN,1.0,NaN,8.0,NaN,0.247121,1,0,0
11172,669,1971,17,Head,683.0,0.0,3439.611816,NaN,0,NaN,...,66.0,NaN,2.0,NaN,12.0,NaN,0.247121,1,0,0
11173,284,1971,20,Head,2400.0,0.0,76885.437500,NaN,2,1.0,...,39.0,36.0,1.0,2.0,16.0,12.0,0.247121,1,0,0


In [198]:
#Dropping 0 values from the hours variable to avoid generating infinite values for wage variable
df_new['hours']=df_new['hannhrs'].where(df_new['hannhrs'] > 0)

In [199]:
#Creating an hourly wage variable
df_new['Wage'] = df_new['hlabinc']/df_new['hours'] 

In [200]:
df_new.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,hpersno,wpersno,Educ,wyrsed,pce,White,Black,Other,hours,Wage
count,89727.000000,89727.000000,89727.000000,89727.000000,89727.000000,8.972700e+04,45372.000000,89727.000000,73851.000000,89688.000000,...,89727.000000,62744.000000,89727.000000,62283.000000,89727.000000,89727.000000,89727.000000,89727.000000,89537.000000,89537.000000
mean,1510.582500,1986.321453,3515.416474,2067.515625,763.884583,4.211645e+04,21923.435547,0.949692,1.096911,1.123695,...,48.799870,65.535561,13.228849,13.029382,0.609883,0.897913,0.079664,0.021989,2071.903076,20.665855
std,834.683767,8.794182,2313.887855,756.214233,913.208374,4.675379e+04,20679.593750,1.168184,0.354742,0.390376,...,73.757706,81.078194,2.526552,2.225672,0.208727,0.302765,0.270774,0.146648,750.982544,24.840120
min,1.000000,1971.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.247121,0.000000,0.000000,0.000000,1.000000,0.000204
25%,783.000000,1979.000000,1673.000000,1800.000000,0.000000,1.976367e+04,8042.066406,0.000000,1.000000,1.000000,...,1.000000,2.000000,12.000000,12.000000,0.421747,1.000000,0.000000,0.000000,1804.000000,10.413431
50%,1541.000000,1986.000000,3322.000000,2064.000000,75.000000,3.460022e+04,18134.263672,0.000000,1.000000,1.000000,...,4.000000,4.000000,12.000000,12.000000,0.614522,1.000000,0.000000,0.000000,2064.000000,16.558668
75%,2240.000000,1993.000000,5058.500000,2452.000000,1701.000000,5.267309e+04,30188.000977,2.000000,1.000000,1.000000,...,170.000000,170.000000,16.000000,15.000000,0.786908,1.000000,0.000000,0.000000,2456.000000,24.737776
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,685266.750000,11.000000,8.000000,3.000000,...,227.000000,231.000000,17.000000,17.000000,0.928007,1.000000,1.000000,1.000000,7800.000000,1865.037964


In [201]:
#Creating the y variable log of wage
df_new['lnwage'] = np.log(df_new.Wage)

In [202]:
# Selecting the final data that I will use for my analysis. 
#We need only male's that are head of household, that are 25-60 years old and make more than 7$/hour
final= df_new[(df_new['hsex']==1) & (df_new['age'] <= 60) & (df_new['age'] >= 25) & (df_new['Wage'] > 7)]

In [203]:
final.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,wpersno,Educ,wyrsed,pce,White,Black,Other,hours,Wage,lnwage
count,57097.000000,57097.000000,57097.000000,57097.000000,57097.000000,5.709700e+04,37163.000000,57097.000000,51707.000000,57062.000000,...,49810.000000,57097.000000,49460.000000,57097.000000,57097.000000,57097.000000,57097.000000,57097.000000,57097.000000,57097.000000
mean,1507.174860,1986.584129,3480.671944,2228.480713,991.917847,5.282710e+04,23085.300781,1.173337,1.086275,1.101416,...,67.981453,13.529993,13.210190,0.616286,0.920539,0.056343,0.022506,2228.480713,24.320503,3.010798
std,828.407534,8.716500,2253.042974,620.018860,926.230774,5.235579e+04,21002.712891,1.216793,0.343647,0.369015,...,81.709457,2.449510,2.163178,0.206366,0.270460,0.230584,0.148322,620.018860,25.204367,0.544119
min,1.000000,1971.000000,1.000000,2.000000,0.000000,1.666980e+01,1.192780,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.247121,0.000000,0.000000,0.000000,2.000000,7.000252,1.945946
25%,782.000000,1979.000000,1690.000000,1952.000000,0.000000,3.037345e+04,8895.574219,0.000000,1.000000,1.000000,...,2.000000,12.000000,12.000000,0.421747,1.000000,0.000000,0.000000,1952.000000,13.950494,2.635515
50%,1542.000000,1987.000000,3296.000000,2160.000000,955.000000,4.381145e+04,19501.712891,1.000000,1.000000,1.000000,...,4.000000,13.000000,12.000000,0.635834,1.000000,0.000000,0.000000,2160.000000,19.914677,2.991457
75%,2225.000000,1994.000000,5003.000000,2519.000000,1900.000000,6.138394e+04,31769.908203,2.000000,1.000000,1.000000,...,170.000000,16.000000,15.000000,0.803488,1.000000,0.000000,0.000000,2519.000000,27.793240,3.324793
max,2930.000000,2002.000000,16968.000000,5840.000000,5840.000000,3.771521e+06,417271.468750,11.000000,8.000000,3.000000,...,230.000000,17.000000,17.000000,0.928007,1.000000,1.000000,1.000000,5840.000000,1717.330322,7.448526


In [204]:
final['age'].describe()

count    57097.000000
mean        39.242939
std          9.579581
min         25.000000
25%         31.000000
50%         38.000000
75%         47.000000
max         60.000000
Name: age, dtype: float64

In [205]:
#Now splitting up the years, starting with year 1971
year1= final[(final['year']==1971)] 
year1.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,wpersno,Educ,wyrsed,pce,White,Black,Other,hours,Wage,lnwage
count,1380.000000,1380.0,1380.000000,1380.000000,1380.000000,1380.000000,727.000000,1380.000000,1012.000000,1380.000000,...,1284.000000,1380.000000,1272.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000
mean,1520.844928,1971.0,2484.165942,2239.666748,654.630432,46228.089844,15088.083984,1.594928,1.077075,1.120290,...,17.131620,12.210145,11.988208,0.247122,0.906522,0.066667,0.026812,2239.666748,21.360428,2.936990
std,838.157827,0.0,1403.973525,635.049377,817.565491,27814.853516,12095.452148,1.540030,0.314484,0.399448,...,47.952663,3.074726,2.322334,0.000001,0.291207,0.249534,0.161591,635.049377,13.763299,0.471703
min,1.000000,1971.0,1.000000,71.000000,0.000000,590.803894,36.419418,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.247121,0.000000,0.000000,0.000000,71.000000,7.003735,1.946444
25%,795.500000,1971.0,1313.750000,1946.250000,0.000000,29916.529297,4451.262207,0.000000,1.000000,1.000000,...,2.000000,11.000000,11.000000,0.247121,1.000000,0.000000,0.000000,1946.250000,13.691573,2.616780
50%,1569.500000,1971.0,2399.000000,2136.000000,80.000000,40901.029297,13353.787109,1.000000,1.000000,1.000000,...,2.000000,12.000000,12.000000,0.247121,1.000000,0.000000,0.000000,2136.000000,18.630970,2.924825
75%,2251.250000,1971.0,3695.500000,2501.000000,1440.000000,56652.429688,23569.432617,2.000000,1.000000,1.000000,...,2.000000,14.000000,12.000000,0.247121,1.000000,0.000000,0.000000,2501.000000,25.181152,3.226096
max,2927.000000,1971.0,5057.000000,5140.000000,3000.000000,396567.000000,60699.031250,11.000000,3.000000,3.000000,...,181.000000,17.000000,17.000000,0.247121,1.000000,1.000000,1.000000,5140.000000,230.959808,5.442244


## Maximum Likelihood Estimator

In [216]:
def neg_loglike(beta):
    #Predicted Model
    yhat = beta[0] + beta[1]*year1['Educ'] + beta[2]*year1['age'] + beta[3]*year1['Black'] + beta[4]* year1['Other']
    
    
    
    #Defining the log-likelihood as a negative log likelihood which is a sum of the log of a normal pdf
    #with mean yhat and standard deviation beta[5]
    #All other years will follow a similar process
    
    logl =  -1*stats.norm(yhat, beta[5]).logpdf(year1['lnwage']).sum()
    
    return logl

In [217]:
#Defining the initial guesses for b0-b4 as well as b5 (st.deviation) respectively 

beta_start = np.array([2, -2, 0.5, 0.5, 0.5, 1])

#optimizing the log-likelihood over the given parameters using the Nelder-Mead method

result = minimize(neg_loglike, beta_start, method = 'Nelder-Mead', options={'disp': True})

print(result)


Optimization terminated successfully.
         Current function value: 728.062858
         Iterations: 681
         Function evaluations: 1059
 final_simplex: (array([[ 1.55114748,  0.06686841,  0.0143899 , -0.16395096,  0.03064082,
         0.41009259],
       [ 1.55121174,  0.06687153,  0.01438785, -0.16389915,  0.03063831,
         0.4100891 ],
       [ 1.55108379,  0.06687504,  0.01438969, -0.16392666,  0.0307296 ,
         0.41008253],
       [ 1.55111934,  0.06687736,  0.01438823, -0.16392111,  0.03058678,
         0.41009722],
       [ 1.55115975,  0.06687352,  0.01438812, -0.16392127,  0.03071874,
         0.41009128],
       [ 1.5511094 ,  0.06687689,  0.01438825, -0.16386633,  0.03057076,
         0.41008561],
       [ 1.55115496,  0.06687305,  0.01438857, -0.16394035,  0.0306875 ,
         0.41008532]]), array([ 728.06285792,  728.06285888,  728.06286502,  728.0628697 ,
        728.06287323,  728.0628809 ,  728.06288942]))
           fun: 728.06285791806476
       message: '

In [133]:
res1 =pd.DataFrame({'parameters':res1['x']})
res1.index=['Intercept', 'b1', 'b2', 'b3', 'b4', 'sigma']
res1.head()

,parameters
Intercept,1.551147
b1,0.066868
b2,0.014390
b3,-0.163951
b4,0.030641


In [287]:
#Now looking only at year 1980
year2= final[(final['year']==1980)] 
year2.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,wpersno,Educ,wyrsed,pce,White,Black,Other,hours,Wage,lnwage
count,1856.000000,1856.0,1856.000000,1856.000000,1856.000000,1856.000000,1152.000000,1856.000000,1575.000000,1856.000000,...,1633.000000,1856.000000,1633.000000,1856.000000,1856.000000,1856.000000,1856.000000,1856.000000,1856.000000,1856.000000
mean,1529.302263,1980.0,3218.014009,2187.516602,862.890076,48254.414062,18117.966797,1.199353,1.073651,1.106681,...,55.364971,13.217134,12.764850,0.466735,0.915409,0.062500,0.022091,2187.516602,22.684633,2.986743
std,828.231155,0.0,1843.885088,642.458801,883.042175,31733.013672,14426.778320,1.204717,0.308145,0.373573,...,77.920090,2.504283,2.095117,0.000006,0.278346,0.242127,0.147018,642.458801,16.444181,0.492932
min,4.000000,1980.0,2.000000,21.000000,0.000000,321.386169,64.277237,0.000000,1.000000,1.000000,...,1.000000,3.000000,3.000000,0.466728,0.000000,0.000000,0.000000,21.000000,7.040302,1.951651
25%,818.750000,1980.0,1681.750000,1920.000000,0.000000,29996.042969,6427.723145,0.000000,1.000000,1.000000,...,2.000000,12.000000,12.000000,0.466728,1.000000,0.000000,0.000000,1920.000000,14.056891,2.643113
50%,1591.000000,1980.0,3243.500000,2112.000000,629.500000,42851.488281,15829.339844,1.000000,1.000000,1.000000,...,3.000000,12.000000,12.000000,0.466728,1.000000,0.000000,0.000000,2112.000000,19.963352,2.993898
75%,2244.000000,1980.0,4800.750000,2483.500000,1800.000000,57849.511719,26570.601562,2.000000,1.000000,1.000000,...,170.000000,16.000000,14.000000,0.466728,1.000000,0.000000,0.000000,2483.500000,26.782181,3.287737
max,2927.000000,1980.0,6615.000000,5824.000000,5011.000000,317866.781250,145358.671875,6.000000,3.000000,3.000000,...,181.000000,17.000000,17.000000,0.466728,1.000000,1.000000,1.000000,5824.000000,446.369659,6.101148


In [288]:
def neg_loglike(beta):
    yhat = beta[0] + beta[1]*year2['Educ'] + beta[2]*year2['age'] + beta[3]*year2['Black'] + beta[4]* year2['Other']
    
    logl =  -1*stats.norm(yhat, beta[5]).logpdf(year2['lnwage']).sum()
    return logl

In [289]:
beta_start = np.array([2, 1, 1, -0.5, 0.5, 1])

res2 = minimize(neg_loglike, beta_start, method = 'Nelder-Mead', options={'disp': True})

print(res2)



Optimization terminated successfully.
         Current function value: 1167.197338
         Iterations: 619
         Function evaluations: 987
 final_simplex: (array([[ 1.62687924,  0.06690139,  0.01221821, -0.02486872,  0.43503576,
         0.45434005],
       [ 1.62689705,  0.06689914,  0.01221922, -0.0249499 ,  0.43507049,
         0.45434535],
       [ 1.62696523,  0.0668925 ,  0.01221956, -0.02491155,  0.43504537,
         0.45434972],
       [ 1.62686832,  0.06690002,  0.01221905, -0.02489276,  0.43504694,
         0.45434866],
       [ 1.62681401,  0.06689965,  0.0122206 , -0.02487658,  0.43504566,
         0.45434485],
       [ 1.62686546,  0.06690174,  0.01221927, -0.02493448,  0.43506732,
         0.45434025],
       [ 1.6267854 ,  0.06690298,  0.01222035, -0.02481188,  0.43501964,
         0.45433987]]), array([ 1167.19733763,  1167.19735172,  1167.19735336,  1167.19735791,
        1167.19736196,  1167.19737136,  1167.19737279]))
           fun: 1167.1973376281089
       mes

In [286]:
res2 =pd.DataFrame({'parameters':res2['x']})
res2.index=['Intercept', 'b1', 'b2', 'b3', 'b4', 'sigma']
res2.head()


,parameters
Intercept,1.626879
b1,0.066901
b2,0.012218
b3,-0.024869
b4,0.435036


In [207]:
#Now looking only at year 1990
year3= final[(final['year']==1990)] 
year3.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,wpersno,Educ,wyrsed,pce,White,Black,Other,hours,Wage,lnwage
count,2013.000000,2013.0,2013.000000,2013.000000,2013.000000,2013.000000,1451.000000,2013.000000,1953.000000,2013.000000,...,1753.000000,2013.000000,1750.000000,2013.000000,2013.000000,2013.000000,2013.000000,2013.000000,2013.000000,2013.000000
mean,1506.764531,1990.0,4586.804272,2275.852539,1152.904175,53131.402344,23856.576172,1.159960,1.083461,1.084451,...,78.557327,13.883756,13.554857,0.721435,0.930452,0.054645,0.014903,2275.852539,23.588531,2.988202
std,822.890325,0.0,2478.085045,592.799988,933.885193,45474.148438,20483.181641,1.183151,0.329085,0.327381,...,83.549347,2.223457,2.066671,0.000004,0.254447,0.227342,0.121195,592.799988,19.707569,0.546725
min,4.000000,1990.0,1.000000,50.000000,0.000000,1108.906982,83.168030,0.000000,1.000000,1.000000,...,1.000000,4.000000,6.000000,0.721431,0.000000,0.000000,0.000000,50.000000,7.000676,1.946007
25%,796.000000,1990.0,2589.000000,1974.000000,0.000000,30148.410156,9344.621094,0.000000,1.000000,1.000000,...,3.000000,12.000000,12.000000,0.721431,1.000000,0.000000,0.000000,1974.000000,13.431529,2.597605
50%,1559.000000,1990.0,4683.000000,2205.000000,1300.000000,44356.281250,20792.007812,1.000000,1.000000,1.000000,...,6.000000,14.000000,13.000000,0.721431,1.000000,0.000000,0.000000,2205.000000,19.541361,2.972533
75%,2212.000000,1990.0,6572.000000,2550.000000,1960.000000,62376.019531,33267.210938,2.000000,1.000000,1.000000,...,171.000000,16.000000,16.000000,0.721431,1.000000,0.000000,0.000000,2550.000000,27.930822,3.329731
max,2927.000000,1990.0,9345.000000,5200.000000,4368.000000,762373.562500,277226.750000,7.000000,3.000000,3.000000,...,191.000000,17.000000,17.000000,0.721431,1.000000,1.000000,1.000000,5200.000000,317.283844,5.759797


In [272]:
def neg_loglike(beta):
    yhat = beta[0] + beta[1]*year3['Educ'] + beta[2]*year3['age'] + beta[3]*year3['Black'] + beta[4]* year3['Other']
    
    logl =  -1*stats.norm(yhat, beta[5]).logpdf(year3['lnwage']).sum()
    return logl

In [273]:
beta_start = np.array([2, -1, 0.5, 0.5, 0.5, 1])

res3 = minimize(neg_loglike, beta_start, method = 'Nelder-Mead', options={'disp': True})

print(res3)


Optimization terminated successfully.
         Current function value: 1393.882137
         Iterations: 741
         Function evaluations: 1180
 final_simplex: (array([[ 1.11855319,  0.09755967,  0.01346536, -0.17192547, -0.05960034,
         0.48356114],
       [ 1.11860257,  0.09755539,  0.01346572, -0.17194051, -0.05964875,
         0.48355992],
       [ 1.1185606 ,  0.09755932,  0.01346507, -0.17192997, -0.05962425,
         0.48356525],
       [ 1.11855597,  0.09755674,  0.01346629, -0.1719274 , -0.059562  ,
         0.48356224],
       [ 1.11857498,  0.09755749,  0.0134655 , -0.17190021, -0.0596607 ,
         0.48356244],
       [ 1.11856726,  0.0975581 ,  0.01346565, -0.17190866, -0.05964049,
         0.48356341],
       [ 1.11852874,  0.09756009,  0.01346598, -0.17191742, -0.05964425,
         0.4835614 ]]), array([ 1393.88213727,  1393.88215439,  1393.88215792,  1393.88216019,
        1393.88216186,  1393.8821675 ,  1393.8821763 ]))
           fun: 1393.8821372733109
       me

In [274]:
res3 =pd.DataFrame({'parameters':res3['x']})
res3.index=['Intercept', 'b1', 'b2', 'b3', 'b4', 'sigma']
res3.head()

,parameters
Intercept,1.118553
b1,0.097560
b2,0.013465
b3,-0.171925
b4,-0.059600


In [208]:
#Now looking only at year 2000
year4= final[(final['year']==2000)] 
year4.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,wpersno,Educ,wyrsed,pce,White,Black,Other,hours,Wage,lnwage
count,2595.000000,2595.0,2595.000000,2595.000000,2595.000000,2.595000e+03,1784.000000,2595.000000,2496.000000,2580.000000,...,2154.000000,2595.000000,2081.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000
mean,1496.867823,2000.0,3462.025048,2259.263672,1181.238037,6.384246e+04,31749.779297,0.968786,1.100160,1.118605,...,99.139275,13.901734,13.845266,0.896233,0.909056,0.052408,0.032755,2259.263672,28.443388,3.117217
std,838.342028,0.0,2078.540111,597.081116,976.454346,7.439892e+04,26718.029297,1.118719,0.398876,0.412911,...,82.034149,2.091758,1.975827,0.000023,0.287585,0.222892,0.178030,597.081116,30.680948,0.605962
min,4.000000,2000.0,4.000000,104.000000,0.000000,7.810670e+02,111.580994,0.000000,1.000000,1.000000,...,1.000000,4.000000,8.000000,0.896210,0.000000,0.000000,0.000000,104.000000,7.090258,1.958722
25%,762.000000,2000.0,1691.000000,1980.000000,0.000000,3.347430e+04,15621.339844,0.000000,1.000000,1.000000,...,5.000000,12.000000,12.000000,0.896210,1.000000,0.000000,0.000000,1980.000000,14.949019,2.704646
50%,1520.000000,2000.0,3334.000000,2200.000000,1428.000000,4.686402e+04,26779.439453,1.000000,1.000000,1.000000,...,170.000000,14.000000,14.000000,0.896210,1.000000,0.000000,0.000000,2200.000000,21.401117,3.063443
75%,2245.000000,2000.0,5123.500000,2550.000000,2000.000000,7.141184e+04,40169.160156,2.000000,1.000000,1.000000,...,173.000000,16.000000,16.000000,0.896210,1.000000,0.000000,0.000000,2550.000000,31.446275,3.448278
max,2927.000000,2000.0,7456.000000,4900.000000,4940.000000,1.290657e+06,379375.375000,8.000000,8.000000,3.000000,...,212.000000,17.000000,17.000000,0.896210,1.000000,1.000000,1.000000,4900.000000,526.798950,6.266819


In [269]:
def neg_loglike(beta):
    yhat = beta[0] + beta[1]*year4['Educ'] + beta[2]*year4['age'] + beta[3]*year4['Black'] + beta[4]* year4['Other']
    
    logl =  -1*stats.norm(yhat, beta[5]).logpdf(year4['lnwage']).sum()
    return logl

In [270]:
beta_start = np.array([2, -1, 0.5, -0.5, 0.5, 1])

y2000 = minimize(neg_loglike, beta_start, method = 'Nelder-Mead', options={'disp': True})

print(y2000)


Optimization terminated successfully.
         Current function value: 2081.366950
         Iterations: 753
         Function evaluations: 1181
 final_simplex: (array([[ 1.1609899 ,  0.10922855,  0.01097847, -0.24569339, -0.06044025,
         0.53964669],
       [ 1.16096633,  0.10922863,  0.01097897, -0.24570005, -0.06046799,
         0.53964897],
       [ 1.16101147,  0.10922588,  0.01097885, -0.24570704, -0.06045947,
         0.53965316],
       [ 1.16105346,  0.10922458,  0.01097827, -0.24569697, -0.06047854,
         0.53965011],
       [ 1.16095087,  0.10922982,  0.01097883, -0.24572757, -0.06042966,
         0.53964674],
       [ 1.16103173,  0.10922483,  0.01097864, -0.24569937, -0.06046346,
         0.53964801],
       [ 1.16098802,  0.10922791,  0.01097876, -0.24566238, -0.06049927,
         0.53965617]]), array([ 2081.36695014,  2081.36696865,  2081.3669721 ,  2081.36697846,
        2081.36698915,  2081.36699176,  2081.36699596]))
           fun: 2081.3669501405948
       me

In [271]:
y2000 =pd.DataFrame({'parameters':y2000['x']})
y2000.index=['Intercept', 'b1', 'b2', 'b3', 'b4', 'sigma']
y2000.head()

,parameters
Intercept,1.160990
b1,0.109229
b2,0.010978
b3,-0.245693
b4,-0.060440


## Interpretations

In these examples by year, the coefficient listed next to b1 represents the coefficient on Education. For 1971 a one year increase in education results in an approximate 6.69% increase in earnings. In 1980, a one year increase in education results in a 6.69% increase in earnings. In 1990 this figure has increased to 9.76%. Finally, in the year 2000 a one year increase in education increases expected earnings by 10.92%. From this analysis, the main conclusion is that the effect of education on earnings has increased over time. 